In [1]:
import numpy as np
import os
import tensorflow as tf
from IPython.display import clear_output, Image, display, HTML

###### Do not modify here ###### 
def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = graph_def
    #strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

###### Do not modify here ###### 

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

reset_graph()

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/")

# training on MNIST but only on digits 0 to 4
X_train1 = mnist.train.images[mnist.train.labels < 5]
y_train1 = mnist.train.labels[mnist.train.labels < 5]
X_valid1 = mnist.validation.images[mnist.validation.labels < 5]
y_valid1 = mnist.validation.labels[mnist.validation.labels < 5]
X_test1 = mnist.test.images[mnist.test.labels < 5]
y_test1 = mnist.test.labels[mnist.test.labels < 5]

###### Do not modify here ###### 



Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


## Define Graph

In [2]:
dropout = 0.3

In [3]:
def fully_connected_layer(x, dims, activate, name, dropout=None):
    '''
    [Function Input]
    x        "Tensor" : The input tensor
    dims     "List"   : The dimension of Input and Output
    activate "String" : The name of activate function
    name     "String" : The name of this operation
    
    [Function Output]
    The Result of Linear Function + Activate Function 
    '''
    with tf.variable_scope(name):
#         W = tf.get_variable("Weights", dims, initializer=tf.contrib.layers.variance_scaling_initializer()) # Initialize Weight with He
#         b = tf.get_variable("Bias", [1], initializer=tf.constant_initializer(0)) # Initialize Bias with constant
#         output = tf.matmul(x, W) + b # Connect each elements for model
        
        # Define Activate Function
        if activate == 'elu':
#             output = tf.nn.elu(output, name="Elu")
            activation = tf.nn.elu
        elif activate == 'softmax':
#             output = tf.nn.softmax(output, name="Softmax")
            activation = tf.nn.softmax
        
        #Build Layer
        output = tf.layers.dense(inputs=x, 
                                 units=dims[1], 
                                 activation=activation, 
                                 kernel_initializer=tf.contrib.layers.variance_scaling_initializer(),
                                 name="Neural_Operation"
                                )  # Layer using dense
        if dropout:
            output = tf.layers.dropout(output, rate=dropout, name="Dropout")
    return output


# Clear graph
tf.reset_default_graph()

# Define Input Entry
x = tf.placeholder(tf.float32, shape=(None, 784), name="Input_X") # Input
y = tf.placeholder(tf.int32, shape=(None,), name ="Input_Label")  # Labels

# Define Connect Layers
x1 = fully_connected_layer(x , [784, 128], "elu", "Fully_Connected_Layer_1", dropout) # 1st 128 nodes Layer
x2 = fully_connected_layer(x1, [128, 128], "elu", "Fully_Connected_Layer_2", dropout) # 2nd 128 nodes Layer
x3 = fully_connected_layer(x2, [128, 128], "elu", "Fully_Connected_Layer_3", dropout) # 3rd 128 nodes Layer
x4 = fully_connected_layer(x3, [128, 128], "elu", "Fully_Connected_Layer_4", dropout) # 4th 128 nodes Layer
x5 = fully_connected_layer(x4, [128, 128], "elu", "Fully_Connected_Layer_5", dropout) # 5th 128 nodes Layer
y_ = fully_connected_layer(x5, [128, 5], "softmax", "Softmax_Layer")         # 128 to 5 class with softmax

# Define Loss Function
cross_entropy = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=y_, name="Cross_Entropy"))

# Define Training Process
train_step = tf.train.AdamOptimizer(0.0017).minimize(cross_entropy)

# Define Accuracy
correct_predict = tf.equal(y, tf.argmax(y_,1, output_type=tf.int32)) # [True, False ..., True]
accuracy = tf.reduce_mean(tf.cast(correct_predict, tf.float32)) # [True, False ..., True] --> [1,0,...,1]

In [4]:
with tf.Session() as sess:
    show_graph(tf.get_default_graph().as_graph_def())

In [5]:
train_data = tf.contrib.data.Dataset.from_tensor_slices((X_train1,y_train1)).batch(2048).repeat()
train_iterator = train_data.make_one_shot_iterator()
train_next_batch = train_iterator.get_next()

In [7]:
saver = tf.train.Saver() # to store the model

# Model save criteria
best_accuracy = 0
fail_count = 0

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for epochs in range(3000):
        train, label = sess.run(train_next_batch)
        sess.run(train_step, feed_dict={x:train, y:label})
        
        # Get best accuracy
        if epochs % 100 == 0:
            valid_accuracy = sess.run(accuracy, feed_dict={x:X_valid1, y:y_valid1})
            if valid_accuracy > best_accuracy:
                best_accuracy = valid_accuracy
                print("Best epochs:{} \n\tTrain Accuracy:{}\n\tValid Accuracy:{}\n".format(
                        epochs+1,
                        sess.run(accuracy, feed_dict={x:train, y:label}),
                        valid_accuracy
                    )
                 )
                save_path = saver.save(sess, "./Models/Team26_HW2.ckpt")
            else:
                fail_count += 1
                if fail_count > 5:
                    print("Early Stop! Model seems not improve any more..")
                    break
                
        
        if epochs % 300 == 0:
            print("{} epochs\n\tTrain Accuracy:{}\n\tValid Accuracy:{}\n".format(
                        epochs+1,
                        sess.run(accuracy, feed_dict={x:train, y:label}),
                        valid_accuracy
                    )
                 )
    print("Final\n\tTrain Accuracy:{}\n\tValid Accuracy:{}\n\tTest  Accuracy:{}\n".format(
                        sess.run(accuracy, feed_dict={x:train, y:label}),
                        valid_accuracy,
                        sess.run(accuracy, feed_dict={x:X_test1, y:y_test1})
                    )
                 )
        
    

Best epochs:1 
	Train Accuracy:0.58056640625
	Valid Accuracy:0.6133698225021362

1 epochs
	Train Accuracy:0.58056640625
	Valid Accuracy:0.6133698225021362

Best epochs:101 
	Train Accuracy:0.98486328125
	Valid Accuracy:0.9827989339828491

Best epochs:201 
	Train Accuracy:0.98974609375
	Valid Accuracy:0.9839717149734497

Best epochs:301 
	Train Accuracy:0.99072265625
	Valid Accuracy:0.9867082834243774

301 epochs
	Train Accuracy:0.99072265625
	Valid Accuracy:0.9867082834243774

Best epochs:401 
	Train Accuracy:0.994140625
	Valid Accuracy:0.987490177154541

Best epochs:501 
	Train Accuracy:0.99658203125
	Valid Accuracy:0.9894447922706604

Best epochs:601 
	Train Accuracy:0.99853515625
	Valid Accuracy:0.9906176328659058

601 epochs
	Train Accuracy:0.99853515625
	Valid Accuracy:0.990617573261261

Best epochs:701 
	Train Accuracy:0.9951171875
	Valid Accuracy:0.9917903542518616

901 epochs
	Train Accuracy:0.99853515625
	Valid Accuracy:0.9913994073867798

Best epochs:1001 
	Train Accuracy:0.9